<h1>Keyra allt fyrst með N=1, breyta svo í whatever</h1>

In [10]:
using MLDatasets, Flux, LinearAlgebra, ProgressMeter, Tracker
include("DenseNTK.jl")

### MNIST
function load_MNIST()
    """
    Loading the MNIST dataset.
    10 classes of digits from 0 to 9,
    each with 28x28 pixel dimensions.
    X: Grayscale vector, Y: Correct label.
    """

    X_training, Y_training = MNIST(split = :train)[:]
    X_testing, Y_testing = MNIST(split = :test)[:]
    X_training = Flux.flatten(X_training)
    X_testing = Flux.flatten(X_testing)
    Y_training = Flux.onehotbatch(Y_training, 0:9)
    Y_testing = Flux.onehotbatch(Y_testing, 0:9)
    return X_training, Y_training, X_testing, Y_testing
end

### REMOVE LAST BIAS
function remove_last_bias(model, Jacobian)
    """Removes last bias of model in jacobian, because of 'frozen parameter' """
    lastbias = length(Flux.params(model)[length(Flux.params(model))])
    Jacobian = Jacobian[:, 1:end-lastbias]
    return Jacobian
end

remove_last_bias (generic function with 1 method)

In [21]:
### MODEL
model = Chain(
  DenseNTK(28 * 28, 16, relu),        # 784 x 16 + 16 = 12560 parameters
  DenseNTK(16, 16, relu),             #  16 x 16 + 16 =   272 parameters
  DenseNTK(16, 16, relu),             #  16 x 16 + 16 =   272 parameters
  DenseNTK(16, 10, relu)              #  16 x 10 + 10 =   170 parameters
) 

### DATA
N = 1 # BREYTA ÞESSU Í 3000
x = load_MNIST()[1][:,1:N];

In [22]:
### REIKNA JACOBIAN
l = length(model(x[:,1]))
params, restruct = Flux.destructure(model)
Jacobian = zeros(N*l,length(params))

@time begin
    @showprogress for i = 1:size(x)[2]
        h = (p) -> begin
            mod = restruct(p)
            y = mod(x[:,i])
        end
        d = Tracker.data(Tracker.jacobian(h, params))
        Jacobian[(i-1)*l+1:i*l, :] .= d
    end
    Jacobian = remove_last_bias(model, Jacobian)
end # time ends

Progress: 100%|█████████████████████████████████████████| Time: 0:00:12


 12.669355 seconds (8.77 M allocations: 13.527 GiB, 9.82% gc time, 5.70% compilation time: 13% of which was recompilation)


10000×13264 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.523146  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0       0.0  0.0  0.523146
 ⋮                        ⋮              ⋱       ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0 

In [27]:
### REIKNA KERNEL
K = Jacobian*Jacobian'

10000×10000 Matrix{Float64}:
  0.453399   0.0  0.0  0.0710829  0.0  …  -0.0372873  0.0  0.0   0.116811
  0.0        0.0  0.0  0.0        0.0      0.0        0.0  0.0   0.0
  0.0        0.0  0.0  0.0        0.0      0.0        0.0  0.0   0.0
  0.0710829  0.0  0.0  1.36365    0.0      0.13849    0.0  0.0   0.758531
  0.0        0.0  0.0  0.0        0.0      0.0        0.0  0.0   0.0
  0.0        0.0  0.0  0.0        0.0  …   0.0        0.0  0.0   0.0
  0.0709396  0.0  0.0  0.197512   0.0      0.698132   0.0  0.0  -0.0667574
  0.0        0.0  0.0  0.0        0.0      0.0        0.0  0.0   0.0
  0.0        0.0  0.0  0.0        0.0      0.0        0.0  0.0   0.0
  0.042503   0.0  0.0  0.697571   0.0     -0.218929   0.0  0.0   1.40662
  ⋮                                    ⋱                        
  0.0        0.0  0.0  0.0        0.0      0.0        0.0  0.0   0.0
  0.0        0.0  0.0  0.0        0.0      0.0        0.0  0.0   0.0
  0.100418   0.0  0.0  1.38524    0.0      0.118877   0.0 

In [28]:
eigen(K)